In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from main import TradingBot
import random
from IPython.display import clear_output

tb = TradingBot()

In [2]:
tb.loadData(timePeriod='6m', label='NIFTY 50', forceUpdate=True)
tb.computeSignal()

100%|██████████| 52/52 [00:13<00:00,  3.89it/s]


In [3]:
for stock in tb.data:
    indicator = tb.indicatorCollection[stock]["sri"]
    pattern = indicator.df[indicator.df['candlestick_pattern'] == 'CDL3BLACKCROWS_Bear']
    if len(pattern.index)>0:
        print(stock)

In [4]:
indicator = tb.indicatorCollection['BRITANNIA']["sri"]
# indicator.df[indicator.df['candlestick_pattern'] == 'CDL3BLACKCROWS_Bear']
indicator.showIndicator(100)

In [3]:
from tqdm import tqdm
from datasource import getStockData, getIndexData
from datemodule import getDateRange
from tickers import getStockList, recognizePattern, getIndicesList
from indicators import ChannelBreakoutIndicator, SupportResistanceIndicator

stocks, stockLiveData = getStockList('NIFTY IT')
indices, indexLiveData = getIndicesList()
startDate, endDate = getDateRange('1y')
localData = {}

In [4]:
from jugaad_data.nse import stock_df, index_df, NSELive

index_df(symbol='NIFTY 50', from_date=startDate, to_date=endDate)

,Index Name,INDEX_NAME,HistoricalDate,OPEN,HIGH,LOW,CLOSE
0,Nifty 50,NIFTY 50,2023-07-18,19787.50,19819.45,19690.20,19749.25
1,Nifty 50,NIFTY 50,2023-07-17,19612.15,19731.85,19562.95,19711.45
2,Nifty 50,NIFTY 50,2023-07-14,19493.45,19595.35,19433.50,19564.50
3,Nifty 50,NIFTY 50,2023-07-13,19495.20,19567.00,19385.80,19413.75
4,Nifty 50,NIFTY 50,2023-07-12,19497.45,19507.70,19361.75,19384.30
...,...,...,...,...,...,...,...
305,Nifty 50,NIFTY 50,2022-07-25,16662.55,16706.05,16564.25,16631.00
306,Nifty 50,NIFTY 50,2022-07-22,16661.25,16752.25,16610.90,16719.45
307,Nifty 50,NIFTY 50,2022-07-21,16523.55,16626.95,16483.90,16605.25
308,Nifty 50,NIFTY 50,2022-07-20,16562.80,16588.00,16490.95,16520.85
